In [1]:
from setting import METRIC_TYPE
from os import chdir
chdir('../../DB')
from vector_db import MilvusCollection
from relational_db import SQLiteDB
from towhee import AutoConfig, AutoPipes


search_params = {
    "metric_type": METRIC_TYPE
}

collection_name = 'test'
partition = 'test'
sqlite_db = SQLiteDB(collection_name + '_' + partition)

config = AutoConfig.load_config('sentence_embedding')
config.model = 'average_word_embeddings_glove.6B.300d'
sentence_embedding = AutoPipes.pipeline('sentence_embedding', config=config)


collection = MilvusCollection(collection_name)

/home/kelve/anaconda3/envs/COMP87project/lib/python3.11/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
results = collection.search([sentence_embedding('Monkey D. Luffy').get()[0]], "embedding", search_params, 5, partition_names=[partition], output_fields=["keyword", "context_ids"])[0]

for result in results:
    print("distance: ", result.distance)
    entity = result.entity
    print("found keyword: ", entity.keyword)
    for i, context_id in enumerate(entity.context_ids):
        print("==================================================================")
        print("corresponding context: ", i + 1, ". ", sqlite_db.select(['context'], 'context', f'id = {context_id}')[0][0])
    print('\n')

distance:  1.0000001192092896
found keyword:  Monkey D. Luffy
corresponding context:  1 .  , commonly known simply as "Dragon the Revolutionary", is the father of the Straw Hat Pirates Captain Monkey D. Luffy and the son of the Marines hero Monkey D. Garp. He is the infamous leader and founder of the Revolutionary Army who has been attempting to overthrow the World Government. Not much is known about his activities, background, history, or power. He is the World Government's greatest enemy, and is the most dangerous and most wanted man in the world.
corresponding context:  2 .  The protagonists of the "One Piece" series are all the members of the , a crew of nine pirates captained by Monkey D. Luffy. The crew's number increases throughout the series, as Luffy recruits new members. Once Usopp joins the Straw Hat Pirates, they gain their own ship, the "Going Merry", which is later destroyed and replaced by a larger and more powerful vessel, the "Thousand Sunny" created by their shipwrigh